<div>
    <img src="images/emlyon.png" style="height:60px; float:left; padding-right:10px; margin-top:5px" />
    <span>
        <h1 style="padding-bottom:5px;"> Introduction to Deep Learning </h1>
        <a href="https://masters.em-lyon.com/fr/msc-in-data-science-artificial-intelligence-strategy">[DSAIS]</a> MSc in Data Science & Artificial Intelligence Strategy <br/>
         Paris | © Saeed VARASTEH
    </span>
</div>

<div class="alert-info" style="border-bottom: solid 1px lightgray; background-color:#ece4f5;">
    <img src="images/assignment.png" style="height:60px; float:left; padding-right:10px;" />
    <span style="font-weight:bold; color:#8966b0">
        <h4 style="padding-top:25px;"> FINAL PROJECT </h4>
    </span>
</div>

###  DSAIS Face Recognition Application


In this project, you'll build a facial recognition system to recognize all of your friends.

Face recognition problems generally fall into one of two categories:

- __Face verification__, where you build a model to distinguish your face from others. This is a binary classification problem.
- __Face Recognition__, where you build a model to identify multiple faces. This is a multi-class classification problem.

Imagine a system for the Emlyon building where we want to provide face recognition to allow students to enter the building.

The goal is to fine-tune the classification head of __FaceNet__ to classify all your friends.

The project consists of two phases. 

- __Face Detection:__ In the first phase, you will need to prepare your datasets; you will use the __MTCNN__ model to create cropped face images from your raw images and store them in new directories.

- __Face Recognition:__ The second phase is very similar to what we did with transfer learning. You modify the classifier head of __FaceNet__ and train it on the cropped face images.

You will find FaceNet documentation here: [FaceNet](https://github.com/timesler/facenet-pytorch)

Finally, you can use your trained model to test it on test data.

__Extra step (high bounce!)__:

Build a Python application that shows recognized faces of DSAIS students on video feeds (or real-time images from the camera). 

### Phase 01: Face detection

In this cell, we have decided to not include HEIC format pictures. We tried to install pyheif to take them into account but both of our laptops can't install pyheif. Only 20 photos are concerned. We preferred ignoring those.

In [45]:
import os
from facenet_pytorch import MTCNN
from PIL import Image, UnidentifiedImageError

def process_images(source_folder, target_folder):
    mtcnn = MTCNN(keep_all=True)
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    for person_name in os.listdir(source_folder):
        person_path = os.path.join(source_folder, person_name)
        if os.path.isdir(person_path):
            target_person_path = os.path.join(target_folder, person_name)
            if not os.path.exists(target_person_path):
                os.makedirs(target_person_path)
            
            for photo in os.listdir(person_path):
                photo_path = os.path.join(person_path, photo)
                try:
                    image = Image.open(photo_path).convert('RGB')
                    boxes, _ = mtcnn.detect(image)
                    if boxes is not None and len(boxes) > 0:
                        box = boxes[0]  
                        face = image.crop(box)
                        target_photo_path = os.path.join(target_person_path, os.path.splitext(photo)[0] + ".jpg")
                        face.save(target_photo_path)
                except UnidentifiedImageError:
                    print(f"The file {photo_path} is not supported. Ignored.")
                except Exception as e:
                    print(f"Error in image treatment {photo_path}: {e}")

source_folder = './data/samples'
target_folder = './data/samples_cropped'

process_images(source_folder, target_folder)


Le fichier ./data/samples/Benjamin/IMG_5273.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Benjamin/IMG_5340.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Benjamin/IMG_1711.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Benjamin/IMG_5375.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Benjamin/IMG_4640.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Qiqi/11.JPG n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Qiqi/10.JPG n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Selin/IMG_9301_Original.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Selin/IMG_8609_Original.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./data/samples/Selin/IMG_7857.HEIC n'est pas une image reconnue ou supportée. Ignoré.
Le fichier ./

### Phase 02: Face recognition 

In [46]:
import numpy as np
import os
from PIL import Image
from keras_facenet import FaceNet

embedder = FaceNet()

def load_and_embed_faces(directory):
    embeddings = []
    labels = []
    for person_label in os.listdir(directory):
        person_dir = os.path.join(directory, person_label)
        if not os.path.isdir(person_dir):
            continue
        for filename in os.listdir(person_dir):
            path = os.path.join(person_dir, filename)
            if os.path.isdir(path):
                continue
            image = Image.open(path).convert('RGB')
            image_array = np.asarray(image)
            embedding = embedder.embeddings([image_array])[0]
            embeddings.append(embedding)
            labels.append(person_label)
    return np.asarray(embeddings), np.asarray(labels)

directory = './data/samples_cropped/'

embeddings, labels = load_and_embed_faces(directory)


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 43ms/step


In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)


In [48]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Encoding of labels in numerical values
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# SVM Training
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train_encoded)

# Predict on test
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.92


In [49]:
def recognize_face(face_image, embedder, clf, encoder):
    face_array = np.asarray(face_image)
    embedding = embedder.embeddings([face_array])[0]
    
    prediction = clf.predict([embedding])
    identity = encoder.inverse_transform(prediction)[0]
    
    return identity

In [50]:
import cv2
from mtcnn.mtcnn import MTCNN

detector = MTCNN()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    faces = detector.detect_faces(frame)
    
    for face in faces:
        x, y, width, height = face['box']
        face_image = frame[y:y+height, x:x+width]
        
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        face_image = Image.fromarray(face_image)
        
        identity = recognize_face(face_image, embedder, clf, encoder)
        
        cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)
        cv2.putText(frame, identity, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    cv2.imshow('Frame', frame)
    
    # Quit with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 44ms/step
